James Caldwell, UVA IRA, Feb 2026 <br>
This script loads 3 excel files from qlik (enrollment, admissions, and degrees) and calculates the various groupings for the College Graduate Survey. There are ~700 line items calculated.

To do: fix citizenship unknown, country not listed, country naming check

In [1]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os

# print(os.getcwd())

template = pd.read_csv('CGS-survey-template-2025.csv',
                       encoding="cp1252",
                       skiprows=1)

# Load environment variables
load_dotenv()
enrollment_file_path = os.getenv('enrollment') 
enrollment = pd.read_excel(enrollment_file_path)
degrees_file_path = os.getenv('degrees') 
degrees = pd.read_excel(degrees_file_path)
admissions_file_path = os.getenv('admissions') 
admissions = pd.read_excel(admissions_file_path)

### Process degrees data, sorting then dropping duplicates on SSID
degree_order = ['DOCT', 'MAST', 'EDS', 'CERT']
# Convert the column to a Categorical with that order
degrees["Degree Level"] = pd.Categorical(degrees["Degree Level"], categories=degree_order, ordered=True)
# Now sort by SSID first, then by the categorical degree
degrees.sort_values(by=['Student System ID', "Degree Level"], inplace=True)
degrees.drop_duplicates(subset=['Student System ID'], keep='first', inplace=True)
enrollment.rename(columns={"Primary Degree Level": "Degree Level"}, inplace=True)

### Process admissions data, sorting then dropping duplicates on SSID
degree_order = ['DOCT', 'MAST', 'EDS', 'CERT']
# Convert the column to a Categorical with that order
admissions["Degree Level"] = pd.Categorical(admissions["Degree Level"], categories=degree_order, ordered=True)
admissions["Enrollment Flag"] = pd.Categorical(admissions["Enrollment Flag"], categories=["Y", "N"], ordered=True)
admissions["OfferFlag"] = pd.Categorical(admissions["OfferFlag"], categories=[1, 0], ordered=True)
admissions.sort_values(
    by=["Student System ID", "Enrollment Flag", "Degree Level", "OfferFlag"],
    ascending=[True, True, True, True],
    inplace=True)
admissions.drop_duplicates(subset=['Student System ID'], keep='first', inplace=True)


In [2]:
MASTERS_OTHER = ["MAST", "COS", "EDS", "CERT","SP"]
DOCTORAL = ["DOCT"]

GENDER_MAP = {
    "men": ["M"],
    "women": ["F"],
    "other": ["X", "U"],
    "total": ["M", "F", "X", "U"]
}

In [3]:
def debugging_print(question, sub):
    print('yay')
    print(sub["Student System ID"].nunique())

In [20]:
def count_from_question(question, df):
    sub = df.copy()
    question = question.strip().lower().replace("–", "-").replace("-", "-")
    # print(question)

    # --- First-time vs Total ---
    if "first-time" in question:
        sub = sub[sub["New/Returning/Transfer Status"] == "New"]
    
    # --- Degree level ---
    if "master" in question and "doct" in question:
        pass # keep both
    else:
        if "master" in question:
            sub = sub[sub["Degree Level"].isin(MASTERS_OTHER)]
        if "doct" in question:
            sub = sub[sub["Degree Level"].isin(DOCTORAL)]
        if "certificate" in question:
            sub = sub[sub["Degree Level"] == "CERT"]

    # --- Full-time / Part-time ---
    if "full-time" in question and "part-time" in question:
        pass # keep both
    else:
        if "full-time" in question:
            sub = sub[sub["Full or Part Time Status"] == "Full-Time"]
        if "part-time" in question:
            sub = sub[sub["Full or Part Time Status"] == "Part-Time"]
    
    # --- Gender ---
    gender_question = question.replace("& other", "") # This logic captures "other" gender, but not "other" from "Master's & other"
    gender_question = gender_question.replace("master's and other", "master's")
    if question == "offers of admission master's & other":
        print('here 1')
        debugging_print(question, sub)
        print("gender 2:" + gender_question)
        print(sub.shape)
    for label, codes in GENDER_MAP.items():
        if f" {label}" in gender_question:
            # print(label)
            sub = sub[sub["Gender"].isin(codes)]
    if question == "offers of admission master's & other":
        print('here after gender')
        debugging_print(question, sub)
        print('gender:' + gender_question)
        print(sub.shape)
    # --- Ethnicity ---
    if "hispanic" in question:
        sub = sub[sub["IPEDS Race"] == "Hispanic"]
    if "american" in question:
        sub = sub[sub["IPEDS Race"] == "American Indian or Alaska Native"]
    if "asian" in question:
        sub = sub[sub["IPEDS Race"] == "Asian"]
    if "black" in question:
        sub = sub[sub["IPEDS Race"] == "Black or African American"]
    if "hawaiian" in question:
        sub = sub[sub["IPEDS Race"] == "Native Hawaiian or Other Pacific Islander"]
    if "white" in question:
        sub = sub[sub["IPEDS Race"] == "White"]
    if "two or more" in question:
        sub = sub[sub["IPEDS Race"] == "Multi-Race"]
    if "international" in question:
        sub = sub[sub["IPEDS Race"] == "Non-Resident Alien"]
    if "unknown" in question:
        sub = sub[sub["IPEDS Race"] == "Race and Ethnicity Unknown"]
    if "citizens and permanent" in question:
        sub = sub[sub["IPEDS Race"] != "Non-Resident Alien"]

    # --- Country-based questions ---
    parts = [p.strip() for p in question.split(",")]
    if len(parts) > 2: # 3 part questions are country-based
        possible_country = parts[1] #countries are always the second part of the question
        country_series = df["Foreign Citizen Country Desc"].str.strip().str.lower()
        possible_country_norm = possible_country.strip().lower()

        if possible_country_norm in country_series.unique():
            sub = sub.loc[country_series == possible_country_norm]
        else:
            # print('skipped: ' + question)
            return '-'
    # --- Admissions ---
    if 'offers' in question:
        sub = sub[sub["OfferFlag"] == 1]
    if 'not accepted' in question:
        sub = sub[sub["OfferFlag"] == 0]

    if question == "offers of admission master's & other":
        print('here 1')
        debugging_print(question, sub)
        print(gender_question)
        print(sub.shape)

    if df.equals(sub):
        if "total total" in question:
            pass  # allow total total to be unfiltered
        else:
            # print('skipped: ' + question)
            return '-'
    if sub.empty:
        return 0

    return sub["Student System ID"].nunique()

results = {} 
# ensure row 3 exists
template.loc[3] = -1

for col, question in template.iloc[2, :].items():
    if isinstance(question, str) and "Enrollment" in question:
        template.loc[3, col] = count_from_question(question, enrollment)
    elif isinstance(question, str) and ("Degree" in question or "Degrees" in question or "Certificates" in question):
        # print(question)
        template.loc[3, col] = count_from_question(question, degrees)
    elif isinstance(question, str) and ("Admission" in question or "Applications" in question):
        template.loc[3, col] = count_from_question(question, admissions)
    else: # these should be enrollment section IV and V
        template.loc[3, col] = count_from_question(question, enrollment)  

template_long = template.T

template_long.to_csv('CGS-survey-2025-output2.csv', header=False)


here 1
yay
12317
gender 2:offers of admission master's 
(12317, 8)
here after gender
yay
12317
gender:offers of admission master's 
(12317, 8)
here 1
yay
4665
offers of admission master's 
(4665, 8)
